# Credit Fairness Demo

End-to-end walkthrough: generate synthetic credit data, train GLM / NN / ADV_NN, and evaluate fairness at both default and fixed approval thresholds.


## 1. Introduction

- Latent score `S*` drives true default risk and is race neutral.
- Observed score `S` is biased downward for the protected group (`A=1`).
- Proxy `Z` correlates with race.
- Goal: compare models on accuracy (ROC/PR) and fairness (EO/DP), including a fixed 2% approval rate.


## 2. Imports & Config


In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from dataclasses import replace
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

from src.config import get_default_configs
from src.credit import generate_credit_underwriting_data, train_test_split_df
from src.evaluation.metrics import compute_accuracy_metrics
from src.evaluation.fairness import fairness_metrics, fairness_at_target_rate
from src.models.glm_model import GLMClassifier
from src.models.nn_model import PlainNN, train_plain_nn, predict_proba_plain_nn
from src.models.adv_nn_model import AdvPredictor, train_adv_nn, predict_proba_adv_nn


In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / 'src').exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
print(f"Using project root: {PROJECT_ROOT}")


## 3. Generate synthetic data


In [ ]:
sim_cfg, train_cfg, eval_cfg = get_default_configs()
df_full = generate_credit_underwriting_data(sim_cfg)
df_train, df_test = train_test_split_df(df_full, test_size=0.2, seed=sim_cfg.seed)
print(f"Train size: {len(df_train)}, Test size: {len(df_test)}")
df_train.head()


## 4. Feature preprocessing


In [ ]:
numeric_cols = ["S", "D", "L"]
proxy_col = "Z"

scaler = StandardScaler()
X_train = np.concatenate([
    scaler.fit_transform(df_train[numeric_cols]),
    df_train[[proxy_col]].to_numpy(),
], axis=1).astype(np.float32)
y_train = df_train["Y"].to_numpy(dtype=np.float32)
A_train = df_train["A"].to_numpy(dtype=np.int64)

X_test = np.concatenate([
    scaler.transform(df_test[numeric_cols]),
    df_test[[proxy_col]].to_numpy(),
], axis=1).astype(np.float32)
y_test = df_test["Y"].to_numpy(dtype=np.float32)
A_test = df_test["A"].to_numpy(dtype=np.int64)
X_train.shape, X_test.shape


## 5. Train models


### 5.1 GLM


In [ ]:
glm = GLMClassifier().fit(X_train, y_train)
y_proba_glm = glm.predict_proba(X_test)


### 5.2 Plain NN


In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=sim_cfg.seed, stratify=y_train
)

def build_loader(X, y, batch_size, shuffle=True):
    ds = TensorDataset(torch.from_numpy(X).float(), torch.from_numpy(y).float())
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)

train_loader = build_loader(X_tr, y_tr, train_cfg.batch_size, shuffle=True)
val_loader = build_loader(X_val, y_val, train_cfg.batch_size, shuffle=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
plain_nn = PlainNN(input_dim=X_train.shape[1]).to(device)
train_plain_nn(plain_nn, train_loader, val_loader, train_cfg, device)
y_proba_nn = predict_proba_plain_nn(plain_nn, X_test, device=device)


### 5.3 Adversarial NN


In [ ]:
train_cfg_adv = replace(train_cfg, lambda_adv=0.8)
adv_ds = TensorDataset(
    torch.from_numpy(X_train).float(),
    torch.from_numpy(y_train).float(),
    torch.from_numpy(A_train).long(),
)
adv_loader = DataLoader(adv_ds, batch_size=train_cfg.batch_size, shuffle=True)
adv_model = AdvPredictor(input_dim=X_train.shape[1]).to(device)
train_adv_nn(adv_model, adv_loader, train_cfg_adv, device=device)
y_proba_adv = predict_proba_adv_nn(adv_model, X_test, device=device)


## 6. Metrics at threshold 0.5


In [ ]:
def summarize(model, y_true, y_proba, A_true):
    acc = compute_accuracy_metrics(y_true, y_proba)
    fair = fairness_metrics(y_true, y_proba, A_true, threshold=eval_cfg.threshold)
    return {"model": model, **acc, **fair}

summary_default = pd.DataFrame([
    summarize("GLM", y_test, y_proba_glm, A_test),
    summarize("NN", y_test, y_proba_nn, A_test),
    summarize("ADV_NN", y_test, y_proba_adv, A_test),
])
summary_default


## 7. Fairness at 2% approval


In [ ]:
TARGET_RATE = 0.02

def summarize_fixed(model, y_true, y_proba, A_true):
    fair = fairness_at_target_rate(y_true, y_proba, A_true, TARGET_RATE)
    acc = compute_accuracy_metrics(y_true, y_proba)
    return {"model": model, **acc, **fair}

summary_fixed = pd.DataFrame([
    summarize_fixed("GLM", y_test, y_proba_glm, A_test),
    summarize_fixed("NN", y_test, y_proba_nn, A_test),
    summarize_fixed("ADV_NN", y_test, y_proba_adv, A_test),
])
summary_fixed[["model", "roc_auc", "eo_gap_tpr", "eo_gap_fpr", "dp_diff", "dp_ratio", "threshold", "actual_rate"]]


## 8. Plots


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
for ax, group in zip(axes, [0, 1]):
    ax.hist(y_proba_glm[A_test == group], bins=40, alpha=0.5, label=f"GLM A={group}")
    ax.hist(y_proba_adv[A_test == group], bins=40, alpha=0.5, label=f"ADV A={group}")
    ax.set_title(f"Predicted score distribution (A={group})")
    ax.set_xlabel("Predicted probability")
    ax.legend()
plt.tight_layout()
plt.show()


### Fairness frontier (EO gap vs ROC AUC)


In [ ]:
df_frontier = pd.DataFrame({
    "model": summary_fixed["model"],
    "roc_auc": summary_fixed["roc_auc"],
    "eo_gap_tpr": summary_fixed["eo_gap_tpr"],
})
plt.figure(figsize=(6, 4))
for _, row in df_frontier.iterrows():
    plt.scatter(row["eo_gap_tpr"], row["roc_auc"], label=row["model"])
    plt.annotate(row["model"], (row["eo_gap_tpr"], row["roc_auc"]), xytext=(5,5), textcoords="offset points")
plt.xlabel("EO TPR gap")
plt.ylabel("ROC AUC")
plt.title("Fairness frontier (2% approval)")
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend()
plt.show()


## 9. Summary


In [ ]:
text = (
    "### Key insights
"
    f"- GLM highest ROC AUC ({summary_fixed.loc[summary_fixed.model=='GLM', 'roc_auc'].iat[0]:.3f}) but EO gap {summary_fixed.loc[summary_fixed.model=='GLM', 'eo_gap_tpr'].iat[0]:.3f}, DP ratio {summary_fixed.loc[summary_fixed.model=='GLM', 'dp_ratio'].iat[0]:.2f} at 2% approval.
"
    f"- Plain NN slightly less accurate ({summary_fixed.loc[summary_fixed.model=='NN', 'roc_auc'].iat[0]:.3f}) with modest fairness gains.
"
    f"- ADV_NN maintains accuracy ({summary_fixed.loc[summary_fixed.model=='ADV_NN', 'roc_auc'].iat[0]:.3f}) while reducing EO gap to {summary_fixed.loc[summary_fixed.model=='ADV_NN', 'eo_gap_tpr'].iat[0]:.3f} and DP ratio to {summary_fixed.loc[summary_fixed.model=='ADV_NN', 'dp_ratio'].iat[0]:.2f}."
)
print(text)
